# Standardize Snow Clearance Data
last updated 3/3/24

<a name="read"></a>
# 1. Read Spatially Joined Data

In [1]:
import pandas as pd

In [2]:
df_communities = pd.read_csv("../../data/03-spatially-joined/addresses-to-communities.csv")
df_wards = pd.read_csv("../../data/03-spatially-joined/addresses-to-wards2015.csv")
df_dockets = pd.read_csv("../../data/01-tidied/dockets-to-details.csv")

In [3]:
# four dockets with unknown address are omitted because they're not geocoded
len(df_communities)

1914

In [4]:
# four dockets with unknown address are omitted because they're not geocoded
len(df_wards)

1914

In [5]:
len(df_dockets)

3662

In [6]:
df_dockets['docket'].nunique()

1918

# Merge Geographies

In [7]:
df_communities = df_communities[['docket','cleaned_address','latlong','community']]
df_communities.head()

,docket,cleaned_address,latlong,community
0,21DT001189,3832 S WABASH,"41.8246153,-87.62519449555593",DOUGLAS
1,22DT001565,2831 S MICHIGAN,"41.842898074485504,-87.62367788440045",DOUGLAS
2,22DT001566,250 E 29TH ST,"41.84211911644582,-87.61996275276782",DOUGLAS
3,20DT000917,3100 S INDIANA,"41.838241,-87.622033",DOUGLAS
4,22DT001221,835 E 35TH ST,"41.8310311,-87.6245552",DOUGLAS


In [8]:
df_wards = df_wards[['docket','ward']]
df_wards.dropna(subset=['ward'], inplace=True)
df_wards['ward'] = df_wards['ward'].astype(int).astype(str)
df_wards = df_wards.rename(columns={'ward':'ward_1523'})
df_wards.head()

,docket,ward_1523
0,21DT001240,12
1,22DT000758,12
2,20DT001445,12
3,20DT001104,12
4,20DT001102,12


### merge address data

In [9]:
#left-hand join b/c we want to include records with no ward assignment
df_gis = pd.merge(df_communities,df_wards,on="docket", how = 'left')
df_gis.head()

,docket,cleaned_address,latlong,community,ward_1523
0,21DT001189,3832 S WABASH,"41.8246153,-87.62519449555593",DOUGLAS,3
1,22DT001565,2831 S MICHIGAN,"41.842898074485504,-87.62367788440045",DOUGLAS,3
2,22DT001566,250 E 29TH ST,"41.84211911644582,-87.61996275276782",DOUGLAS,4
3,20DT000917,3100 S INDIANA,"41.838241,-87.622033",DOUGLAS,3
4,22DT001221,835 E 35TH ST,"41.8310311,-87.6245552",DOUGLAS,3


In [10]:
len(df_gis)

1914

### parse lat and long

In [11]:
#parse lat and long
df_gis['lat']=df_gis['latlong'].str.split(',').str[0]
df_gis['long']=df_gis['latlong'].str.split(',').str[1]

### review and clean records with no community assigned

In [12]:
df_gis[df_gis["community"].isna()]

,docket,cleaned_address,latlong,community,ward_1523,lat,long
1913,21DT002747,5450 S 47TH,"41.8654712,-87.7423005",NaN,NaN,41.8654712,-87.7423005


In [13]:
df_gis[df_gis["ward_1523"].isna()]

,docket,cleaned_address,latlong,community,ward_1523,lat,long
1913,21DT002747,5450 S 47TH,"41.8654712,-87.7423005",NaN,NaN,41.8654712,-87.7423005


One address, 5450 S 47TH (docket 21DT002747), is not a valid address but was geocoded at 41.8654712,-87.7423005 at the edge of Lawndale by Roosevelt, which doesn't look right. If it's 5450 W. 47th it'd be in suburban Stickney. So I'm choosing to skip it.

In [14]:
#drop record with invalid address
# index_to_drop = df[df['address'] == '5450 S 47TH'].index
# df = df.drop(index_to_drop)

In [15]:
# manually assign community areas for the other two addresses. strangely this address isn't recognized, and placed in West Garfield if I use BLVD instead of ST
df_gis.loc[df_gis['cleaned_address'] == '300 W WASHINGTON ST', 'community'] = 'LOOP'

<a name="docket"></a>
# Merge Geographies into Detailed Dockets

In [16]:
df_dockets_gis = pd.merge(df_dockets,df_gis,on='docket', how = 'left')

In [17]:
len(df_dockets_gis)

3662

In [18]:
df_dockets_gis['docket'].nunique()

1918

### identify and address missing values before grouping

In [19]:
df_dockets_gis[df_dockets_gis["community"].isna()]

,respondent,docket,violation_num,violation_date,hearing_date,violation_address,violation_desc,case_disposition,fine_amt,dept,cleaned_address,latlong,community,ward_1523,lat,long
3,1157 W ERIE,22DT001387,T000207431,2022-02-08,4/11/2022,100 UNKNOWN,10-8-180 Snow and ice removal.,Not Liable,0.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
1797,CORE POWER YOGA LLC,21DT000877,T000203714,2021-02-17,4/12/2021,100 UNKNOWN,10-8-180 Snow and ice removal.,Liable,150.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
2313,"PTS CANAL INC, C/O MARK W LEUNG",21DT000685,T000200346,2021-02-09,3/29/2021,100 UNKNOWN,10-8-180 Snow and ice removal.,Liable,150.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
2351,ROMA PACKING CO,22DT000756,T000212869,2022-01-11,3/21/2022,100 UNKNOWN,10-8-180 Snow and ice removal.,Liable,150.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
3348,"RAINBOW MOTEL CORP, C/O ROSARIO ORTIZ",21DT002747,T000207541,2021-01-27,8/16/2021,5450 S 47TH,10-8-180 Snow and ice removal.,Default,500.0,TRANPORT,5450 S 47TH,"41.8654712,-87.7423005",NaN,NaN,41.8654712,-87.7423005


In [20]:
df_dockets_gis[df_dockets_gis["ward_1523"].isna()]

,respondent,docket,violation_num,violation_date,hearing_date,violation_address,violation_desc,case_disposition,fine_amt,dept,cleaned_address,latlong,community,ward_1523,lat,long
3,1157 W ERIE,22DT001387,T000207431,2022-02-08,4/11/2022,100 UNKNOWN,10-8-180 Snow and ice removal.,Not Liable,0.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
1797,CORE POWER YOGA LLC,21DT000877,T000203714,2021-02-17,4/12/2021,100 UNKNOWN,10-8-180 Snow and ice removal.,Liable,150.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
2313,"PTS CANAL INC, C/O MARK W LEUNG",21DT000685,T000200346,2021-02-09,3/29/2021,100 UNKNOWN,10-8-180 Snow and ice removal.,Liable,150.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
2351,ROMA PACKING CO,22DT000756,T000212869,2022-01-11,3/21/2022,100 UNKNOWN,10-8-180 Snow and ice removal.,Liable,150.0,TRANPORT,NaN,NaN,NaN,NaN,NaN,NaN
3348,"RAINBOW MOTEL CORP, C/O ROSARIO ORTIZ",21DT002747,T000207541,2021-01-27,8/16/2021,5450 S 47TH,10-8-180 Snow and ice removal.,Default,500.0,TRANPORT,5450 S 47TH,"41.8654712,-87.7423005",NaN,NaN,41.8654712,-87.7423005


In [21]:
df_dockets_gis[df_dockets_gis["dept"].isna()]

,respondent,docket,violation_num,violation_date,hearing_date,violation_address,violation_desc,case_disposition,fine_amt,dept,cleaned_address,latlong,community,ward_1523,lat,long
3661,CTLTC 008002384127,24DT000388,T000215624,2023-01-17,5/6/2024,359 N CANAL ST,10-8-180 Snow and ice removal.,Pending,0.0,NaN,359 N CANAL ST,"41.888710862028105,-87.63987180649657",NEAR WEST SIDE,42,41.888710862028105,-87.63987180649657


In [22]:
df_dockets_gis['dept']=df_dockets_gis['dept'].fillna('unknown')
df_dockets_gis['ward_1523']=df_dockets_gis['ward_1523'].fillna('unknown')
df_dockets_gis['community']=df_dockets_gis['community'].fillna('unknown')
df_dockets_gis['lat']=df_dockets_gis['lat'].fillna('unknown')
df_dockets_gis['long']=df_dockets_gis['long'].fillna('unknown')

# Docket-Level Summary
I expect 1918 dockets

In [23]:
# by docket
df_dockets_summary=df_dockets_gis.groupby(['docket','dept','violation_address','lat','long','community','ward_1523','violation_date']).agg(
    n_records=('docket','count'),
).reset_index()

In [24]:
df_dockets_summary.head()

,docket,dept,violation_address,lat,long,community,ward_1523,violation_date,n_records
0,19DS68300L,STRTSAN,4710 S WESTERN AVE,41.807859,-87.68479703503766,BRIGHTON PARK,15,2019-11-13,2
1,19DS69216L,STRTSAN,1425 W MORSE AVE,42.0074513,-87.6668285,ROGERS PARK,49,2019-11-13,1
2,19DS70010L,STRTSAN,715 E 47TH ST,41.8093383,-87.6080127,GRAND BOULEVARD,4,2019-11-13,1
3,19DS72153L,STRTSAN,300 W WASHINGTON ST,41.8818694,-87.7401431,WEST GARFIELD PARK,28,2019-11-12,5
4,19DS72160L,STRTSAN,6929 N SHERIDAN RD,41.9598134,-87.654693,UPTOWN,46,2019-11-14,1


In [25]:
len(df_dockets_summary)

1918

# 5. Export

In [26]:
df_gis.to_csv("../../data/04-standardized/dockets-to-gis.csv", index=False)
df_dockets_gis.to_csv("../../data/04-standardized/dockets-to-details-gis.csv", index= False)
df_dockets_summary.to_csv("../../data/04-standardized/dockets-summary.csv", index= False)